# Black-Litterman Model

In [11]:
import pandas as pd
import numpy as np
from numpy import matrix, array, zeros, empty, sqrt, ones, dot, append, cov, transpose, linspace
from numpy.linalg import inv, pinv
from pylab import *
import scipy.optimize
import random
from pypfopt.risk_models import CovarianceShrinkage


####################################
# Helper Functions
####################################


# Function loads historical stock prices of nine major S&P companies and returns them together
# with their market capitalizations, as of 2013-07-01

def importExcel_CalReturns(full_names, filePath, paths):
    list_of_etf=[]
    # Read the file from its path
    xls = pd.ExcelFile(filePath)
    # Get the content of each spreadsheet and concate them together
    for sheet_name in xls.sheet_names:
        sheet_name = xls.parse(sheet_name)
        list_of_etf.append(sheet_name)
    for e in list_of_etf:
        e['Date'] = pd.to_datetime(e['Date'])
        e.set_index('Date', inplace = True)

    indian_etf = pd.concat(list_of_etf,axis=1,keys=xls.sheet_names)
    # Only keep the close price
    indian_etf_close = indian_etf.xs(key = 'CLOSE', axis=1, level=1)
    indian_etf_close=indian_etf_close[full_names]
    
    indian_etf_close_new=multi_csvs_newdata(full_names, paths)
    indian_etf_close=indian_etf_close.append(indian_etf_close_new)
    # Fill all the NaN with the previous price 
    indian_etf_close.fillna(method='ffill',inplace = True)
        
    return indian_etf_close

def multi_csvs_newdata(full_names, paths):
    pHNGSNGBEES,pICICINIFTY,pM100,pN100,pSETFNN50,pICICISENSX=paths[0],paths[1],paths[2],paths[3],paths[4],paths[5]
    HNGSNGBEES = pd.read_csv(pHNGSNGBEES)
    ICICINIFTY = pd.read_csv(pICICINIFTY)
    ICICISENSX = pd.read_csv(pICICISENSX)
    M100 = pd.read_csv(pM100)
    N100 = pd.read_csv(pN100)
    SETFNN50 = pd.read_csv(pSETFNN50)  
    list_of_etf=[HNGSNGBEES, ICICINIFTY, M100, N100,SETFNN50,ICICISENSX]
    
    for e in list_of_etf:
        e['Date'] = pd.to_datetime(e['Date'])
        e.set_index('Date', inplace = True)
    indian_etf = pd.concat(list_of_etf,axis=1,keys=full_names)
    # Only keep the close price
    indian_etf_close = indian_etf.xs(key = 'Close Price' , axis=1, level=1)

    return indian_etf_close

# Function takes historical stock prices together with market capitalizations and calculates
# names       - array of assets' names
# prices      - array of historical (daily) prices
# caps        - array of assets' market capitalizations
# returns:
# names       - array of assets' names
# weights     - array of assets' weights (derived from mkt caps)
# expreturns  - expected returns based on historical data
# covars          - covariance matrix between assets based on historical data
def assets_meanvar(prices, caps):
#     prices = matrix(prices.T)                       # create numpy matrix from prices
    weights = array(caps) / sum(caps)       # create weights

    # create matrix of historical returns
    rows, cols = prices.shape
    
    returns = empty([rows, cols-1])
    for r in range(rows):
        for c in range(cols-1):
            p0, p1 = prices[r,c], prices[r,c+1]
            returns[r,c] = (p1/p0)-1   

    # calculate expected returns
    expreturns = array([])
    for r in range(rows):
        expreturns = append(expreturns, np.mean(returns[r]))
    # calculate covariances
#     covars=CovarianceShrinkage(prices.T).ledoit_wolf()
    covars = cov(returns)
    
#     covars = covars * 252                          # Annualize covariances

    expreturns = (1+expreturns)**252-1      # Annualize expected returns
    

    return weights, expreturns, covars

#       rf              risk free rate
#       lmb             lambda - risk aversion coefficient
#       C               assets covariance matrix
#       V               assets variances (diagonal in covariance matrix)
#       W               assets weights
#       R               assets returns
#       mean    portfolio historical return
#       var             portfolio historical variance
#       Pi              portfolio equilibrium excess returns
#       tau     scaling factor for Black-litterman
     


# Calculates portfolio mean return
def port_mean(W, R):
    return sum(R*W)

# Calculates portfolio variance of returns
def port_var(W, C):
    return dot(dot(W, C), W)

# Combination of the two functions above - mean and variance of returns calculation
def port_mean_var(W, R, C):
    return port_mean(W, R), port_var(W, C)

# Given risk-free rate, assets returns and covariances, this function calculates
# mean-variance frontier and returns its [x,y] points in two arrays
def solve_frontier(R, C, rf):
    def fitness(W, R, C, r):
        # For given level of return r, find weights which minimizes
        # portfolio variance.
        mean, var = port_mean_var(W, R, C)
        # Big penalty for not meeting stated portfolio return effectively serves as optimization constraint
        penalty = 50*abs(mean-r)
        return var + penalty
    frontier_mean, frontier_var, frontier_weights = [], [], []
    n = len(R)      # Number of assets in the portfolio
    for r in linspace(min(R), max(R), num=20): # Iterate through the range of returns on Y axis
        W = ones([n])/n         # start optimization with equal weights
        b_ = [(0,1) for i in range(n)]
        c_ = ({'type':'eq', 'fun': lambda W: sum(W)-1. })
        optimized = scipy.optimize.minimize(fitness, W, (R, C, r), method='SLSQP', constraints=c_, bounds=b_)
        if not optimized.success:
                raise BaseException(optimized.message)
        # add point to the min-var frontier [x,y] = [optimized.x, r]
        frontier_mean.append(r)                                                 # return
        frontier_var.append(port_var(optimized.x, C))   # min-variance based on optimized weights
        frontier_weights.append(optimized.x)
    return array(frontier_mean), array(frontier_var), frontier_weights

# Given risk-free rate, assets returns and covariances, this
# function calculates weights of tangency portfolio with respect to
# sharpe ratio maximization
def solve_weights(R, C, rf):
    def fitness(W, R, C, rf):
        mean, var = port_mean_var(W, R, C)      # calculate mean/variance of the portfolio
        util = (mean - rf) / sqrt(var)          # utility = Sharpe ratio
        return 1/util                                           # maximize the utility, minimize its inverse value
    n = len(R)
    W = ones([n])/n                                         # start optimization with equal weights
    b_ = [(0.,1.) for i in range(n)]        # weights for boundaries between 0%..100%. No leverage, no shorting
    c_ = ({'type':'eq', 'fun': lambda W: sum(W)-1. })       # Sum of weights must be 100%
    optimized = scipy.optimize.minimize(fitness, W, (R, C, rf), method='SLSQP', constraints=c_, bounds=b_)
    if not optimized.success:
        raise BaseException(optimized.message)
    return optimized.x

def print_assets(names, W, R, C):
    print("%-10s %6s %6s %6s %s" % ("Name", "Weight", "Return", "Dev", "   Correlations"))
    for i in range(len(names)):
        print("%-10s %5.1f%% %5.1f%% %5.1f%%    " % (names[i], 100*W[i], 100*R[i], 100*C[i,i]**.5), end='')
        for j in range(i+1):
            corr = C[i,j] / (sqrt(C[i,i]) * (sqrt(C[j,j]))) # calculate correlation from covariance
            print("%.3f " % corr, end='')
        print()

def optimize_and_display(title, names, R, C, rf, color='black'):
    # optimize
    W = solve_weights(R, C, rf)
    mean, var = port_mean_var(W, R, C)                                              # calculate tangency portfolio
    f_mean, f_var, f_weights = solve_frontier(R, C, rf)             # calculate min-var frontier

    # display min-var frontier
#     print(title)
#     print_assets(names, W, R, C)
    return W
#     n = len(names)
#     scatter([C[i,i]**.5 for i in range(n)], R, marker='x',color=color)  # draw assets
#     for i in range(n):                                                                              # draw labels
#         text(C[i,i]**.5, R[i], '  %s'%names[i], verticalalignment='center', color=color)
#     scatter(var**.5, mean, marker='o', color=color)                 # draw tangency portfolio
#     plot(f_var**.5, f_mean, color=color)                                    # draw min-var frontier
#     xlabel('$\sigma$'), ylabel('$r$')
#     grid(True)
#     show()

    # Display weights
#     m = empty([n, len(f_weights)])
#     for i in range(n):
#           for j in range(m.shape[1]):
#                   m[i,j] = f_weights[j][i]
#     stackplot(f_mean, m)
#     show()

# given the pairs of assets, prepare the views and link matrices. This function is created just for users' convenience
def prepare_views_and_link_matrix(names, views):
    r, c = len(views), len(names)
    Q = [views[i][3] for i in range(r)]     # view matrix
    P = zeros([r, c])                                       # link matrix
    nameToIndex = dict()
    for i, n in enumerate(names):
        nameToIndex[n] = i
    for i, v in enumerate(views):
        name1, name2 = views[i][0], views[i][2]
        P[i, nameToIndex[name1]] = +1 if views[i][1]=='>' else -1
        P[i, nameToIndex[name2]] = -1 if views[i][1]=='>' else +1
    return array(Q), P

####################################
# Main
####################################

#############################################################################################################################
# Common for all selections and time
#############################################################################################################################
filePath='D:/Graduate/ARATZ CAPITAL LLC/data/index11.02.19-2.xlsx'

# list_of_etf=[GOLDBEES,HNGSNGBEES, ICICINIFTY, LICNETFGSC,M100, N100,SETFNN50,ICICISENSX]
# list_of_etf_names=['GOLDBEES','HNGSNGBEES', 'ICICINIFTY', 'LICNETFGSC','M100', 'N100','SETFNN50','ICICISENSX']
full_names= ['HNGSNGBEES', 'ICICINIFTY', 'M100', 'N100','SETFNN50','ICICISENSX']
pHNGSNGBEES,pICICINIFTY,pM100,pN100,pSETFNN50,pICICISENSX=None,None,None,None,None,None,
paths=[pHNGSNGBEES,pICICINIFTY,pM100,pN100,pSETFNN50,pICICISENSX]

today="26-05-2019"
for name,j in zip(full_names,range(len(paths))):
    paths[j]="D:/Graduate/ARATZ CAPITAL LLC/data/12-02-2019-TO-"+today+name+"ALLN.csv"

mkt_caps={'HNGSNGBEES':652933.05,
          'ICICISENSX':278677.18,
          'M100':17696.05,
          'SETFNN50':44715.44,
          'ICICINIFTY':231527.68, 
          'N100':1491558.15}

prices_df=importExcel_CalReturns(full_names, filePath,paths)['2015-03-27':]

#############################################################################################################################
# Select time 
#############################################################################################################################
# startTime='2018-09-01'
# prices=prices[startTime:]


#############################################################################################################################
# Select ETFs 
#############################################################################################################################
etfs5=['HNGSNGBEES', 'ICICINIFTY', 'M100', 'SETFNN50','ICICISENSX']
etfs4=['ICICINIFTY', 'M100', 'SETFNN50','ICICISENSX']
etf_lists=[full_names,etfs5,etfs4]

final_weights=pd.DataFrame([],columns=full_names)


for names in etf_lists:
    caps=[]
    for i in names:
        caps.append(mkt_caps[i])
    prices = matrix(prices_df[names].T)                       # create numpy matrix from prices
    
    # Estimate assets's expected return and covariances
    W, R, C = assets_meanvar(prices, caps)
    rf = 0.0735    # Risk-free rate

    #######################
    # print("Historical Weights")
    # print_assets(names, W, R, C)
    #######################

    # Calculate portfolio historical return and variance
    mean, var = port_mean_var(W, R, C)

    # # Mean-Variance Optimization (based on historical returns)
    optimize_and_display('Optimization based on Historical returns', names, R, C, rf, color='red')
    show()

    # # Black-litterman reverse optimization
    lmb = (mean - rf) / var                         # Calculate return/risk trade-off
    Pi = dot(dot(lmb, C), W)                        # Calculate equilibrium excess returns

    # # Mean-variance Optimization (based on equilibrium returns)
    optimize_and_display('Optimization based on Equilibrium returns', names, Pi+rf, C, rf, color='green')
    show()

    # # Determine views to the equilibrium returns and prepare views (Q) and link (P) matrices
    views = [
            ('SETFNN50', '>', 'ICICISENSX', 0.02),
            ('ICICISENSX', '<', 'M100', 0.02)
            ]

    Q, P = prepare_views_and_link_matrix(names, views)

    #######################
    #     print('Views Matrix Q')
    #     print(Q)
    #     print('Link Matrix P')
    #     print(P)
    #######################

    tau = .025 # scaling factor

    # Calculate omega - uncertainty matrix about views
    omega = dot(dot(dot(tau, P), C), transpose(P)) # 0.025 * P * C * transpose(P)
    # Calculate equilibrium excess returns with views incorporated
    sub_a = inv(dot(tau, C))
    sub_b = dot(dot(transpose(P), inv(omega)), P)
    sub_c = dot(inv(dot(tau, C)), Pi)
    sub_d = dot(dot(transpose(P), inv(omega)), Q)
    Pi = dot(inv(sub_a + sub_b), (sub_c + sub_d))

    # Mean-variance Optimization (based on equilibrium returns)
    weight=optimize_and_display('Optimization based on Equilibrium returns with adjusted views', names, Pi+rf, C, rf, color='blue')
    show()
    final_weights=final_weights.append(pd.DataFrame([weight],columns=names))
    final_weights=final_weights.append(pd.DataFrame([[np.nan]*6], columns=full_names))

arrays=[[today]*6,['ALL 6 ETF','','W/O N100','','W/O N100 and HangSeng',''],['Allocation','']*3]
tuples = list(zip(*arrays))
final_weights.index=pd.MultiIndex.from_tuples(tuples, names=['Date','Portfolio','Metric'])   
final_weights

HNGSNGBEES  ICICINIFTY  \
Date       Portfolio             Metric                               
26-05-2019 ALL 6 ETF             Allocation    0.241614    0.084801   
                                                    NaN         NaN   
           W/O N100              Allocation    0.534018    0.188897   
                                                    NaN         NaN   
           W/O N100 and HangSeng Allocation         NaN    0.404008   
                                                    NaN         NaN   

                                             ICICISENSX      M100     N100  \
Date       Portfolio             Metric                                      
26-05-2019 ALL 6 ETF             Allocation    0.046495  0.041409  0.54817   
                                                    NaN       NaN      NaN   
           W/O N100              Allocation    0.094059  0.111013      NaN   
                                                    NaN       NaN      NaN   
           W/O N100 and HangSeng Allocation    0.230164  0.252914      NaN   
                                                    NaN       NaN      NaN   

                                             SETFNN50  
Date       Portfolio             Metric                
26-05-2019 ALL 6 ETF             Allocation  0.037511  
                                                  NaN  
           W/O N100              Allocation  0.072014  
                                                  NaN  
           W/O N100 and HangSeng Allocation  0.112913  
                                                  NaN

# Mean-Variance Optimization